# The Way to a City's Stomach...

How much can we tell about an area in a city based on the restaurants it has? That is the goal of this project.

## Introduction

People of varying backgrounds, economic statuses, and tastes will shape the restaurant options located near them. So, it would seem likely that the different neighborhoods in a city would be composed of different sorts of food establishments, as well as different cities. This project investigates how closely established city neighborhoods align with clusters determined by restaurants and what sorts of restaurants are present in the various clusters. It also compares the restaurant compositions of different cities.

## Data

Foursquare's API has a search features that allows the user to search for venues in a city which belong to certain categories. Among these categories are Food and Bars. Each of these categories is subdivided into smaller subcategories such as Afghan Restaurant, Burger Joint, etc. Given a city, we can consider all of its restaurants and the corresponding list of which categories it belongs to. Using this data, we can hope to understand the "culinary neighborhoods" of a city. 

In [46]:
# imports

import numpy as np
import pandas as pd
import requests
import json
import folium

from collections import defaultdict

In [4]:
# get foursquare creds

with open('foursquare_api_key.json', 'r') as f:
    api_info = json.loads(f.read())
    client_id = api_info['client_id']
    client_secret = api_info['client_secret']
    v = api_info['v']

In [43]:
radius = 10000
offset = 200
limit = 50
cats = '4d4b7105d754a06374d81259,4bf58dd8d48988d116941735'
city = 'Chicago'

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&near={}&radius={}&categoryId={}&offset={}&limit={}'.format(
    client_id,
    client_secret,
    v,
    city,
    radius,
    cats,
    offset,
    limit
)

print(url)
response = requests.get(url)

https://api.foursquare.com/v2/venues/explore?client_id=ECA125Q5GW40QLFVMGKKZSJWXGUY0YREX45PHZ1HZBLFOEVI&client_secret=ORO1ROD0TWBASJQQDWK2NPKP3JHNK23LOMTUEMUOEUCPIFTV&v=20180602&near=Chicago&radius=10000&categoryId=4d4b7105d754a06374d81259,4bf58dd8d48988d116941735&offset=200&limit=50


In [44]:
result = json.loads(response.content)['response']
total_results = int(result['totalResults'])
result['groups'][0]['items']

restaurants_id_set = set()

def get_restaurant_info(group_item):
    rest_id = group_item['venue']['id']
    name = group_item['venue']['name']
    lat = group_item['venue']['location']['lat']
    lng = group_item['venue']['location']['lng']
    cat = group_item['venue']['categories'][0]['name']
    return (rest_id, name, lat, lng, cat)
    
#my_list = []
for item in result['groups'][0]['items']:
    my_list.append(get_restaurant_info(item))

In [45]:
print(len(my_list))
for entry in my_list:
    print(entry)


248
('594dc6af9e0d544d10309eac', 'Haisous Vietnamese Kitchen', 41.85791043737854, -87.65333144462963, 'Vietnamese Restaurant')
('59f51dc9c0f16338d7fa78cf', 'Ca Phe Da', 41.858011634119485, -87.65360133466521, 'Vietnamese Restaurant')
('59f24e4135d3fc2f62bcecd0', 'S.K.Y.', 41.85786076872874, -87.65781234154781, 'New American Restaurant')
('546795a4498e0e0479f82c4f', 'Azul 18', 41.8580501, -87.6580224, 'Latin American Restaurant')
('56d1168ecd101bd7e0ae6060', 'Little Sheep Mongolian Hot Pot', 41.849700422181655, -87.63196762557331, 'Chinese Restaurant')
('569aa92d498eb5f34d655e78', 'Canton Regio', 41.857971550034044, -87.66425762356093, 'Mexican Restaurant')
('4b38368ff964a520e34c25e3', "Ricobene's", 41.84548308263435, -87.63389230631613, 'Pizza Place')
('5455809d498ec012db68a1f1', 'Qing Xiang Yuan Dumpling', 41.85507202582821, -87.63195045647198, 'Asian Restaurant')
('49bac1c8f964a520a0531fe3', "Manny's Cafeteria & Delicatessen", 41.86793066712168, -87.64199136595965, 'Sandwich Place')


In [50]:
my_map = folium.Map(location = [41.93619834291129, -87.65419057638928], zoom_start = 11)

for entry in my_list:
    folium.Marker(location=[entry[2],entry[3]], popup = entry[1]).add_to(my_map)
my_map